In [ ]:
import random

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.collections as mcoll
import matplotlib.colors as mcolors
from matplotlib.widgets import Button
import json as js

TOLERANCE = 0.15

def dist(point1, point2):
    return np.sqrt(np.power(point1[0] - point2[0], 2) + np.power(point1[1] - point2[1], 2))

class _Button_callback(object):
    def __init__(self, scenes):
        self.i = 0
        self.scenes = scenes
        self.adding_points = False
        self.added_points = []
        self.adding_lines = False
        self.added_lines = []
        self.adding_rects = False
        self.added_rects = []

    def set_axes(self, ax):
        self.ax = ax

    def next(self, event):
        self.i = (self.i + 1) % len(self.scenes)
        self.draw(autoscaling = True)

    def prev(self, event):
        self.i = (self.i - 1) % len(self.scenes)
        self.draw(autoscaling = True)

    def add_point(self, event):
        self.adding_points = not self.adding_points
        self.new_line_point = None
        if self.adding_points:
            self.adding_lines = False
            self.adding_rects = False
            self.added_points.append(PointsCollection([]))

    def add_line(self, event):
        self.adding_lines = not self.adding_lines
        self.new_line_point = None
        if self.adding_lines:
            self.adding_points = False
            self.adding_rects = False
            self.added_lines.append(LinesCollection([]))

    def add_rect(self, event):
        self.adding_rects = not self.adding_rects
        self.new_line_point = None
        if self.adding_rects:
            self.adding_points = False
            self.adding_lines = False
            self.new_rect()

    def new_rect(self):
        self.added_rects.append(LinesCollection([]))
        self.rect_points = []

    def on_click(self, event):
        if event.inaxes != self.ax:
            return
        new_point = (event.xdata, event.ydata)
        if self.adding_points:
            self.added_points[-1].add_points([new_point])
            self.draw(autoscaling = False)
        elif self.adding_lines:
            if self.new_line_point is not None:
                self.added_lines[-1].add([self.new_line_point, new_point])
                self.new_line_point = None
                self.draw(autoscaling = False)
            else:
                self.new_line_point = new_point
        elif self.adding_rects:
            if len(self.rect_points) == 0:
                self.rect_points.append(new_point)
            elif len(self.rect_points) == 1:
                self.added_rects[-1].add([self.rect_points[-1], new_point])
                self.rect_points.append(new_point)
                self.draw(autoscaling = False)
            elif len(self.rect_points) > 1:
                if dist(self.rect_points[0], new_point) < (np.mean([self.ax.get_xlim(), self.ax.get_ylim()])*TOLERANCE):
                    self.added_rects[-1].add([self.rect_points[-1], self.rect_points[0]])
                    self.new_rect()
                else:
                    self.added_rects[-1].add([self.rect_points[-1], new_point])
                    self.rect_points.append(new_point)
                self.draw(autoscaling = False)

    def draw(self, autoscaling = True):
        if not autoscaling:
            xlim = self.ax.get_xlim()
            ylim = self.ax.get_ylim()
        self.ax.clear()
        for collection in (self.scenes[self.i].points + self.added_points):
            if len(collection.points) > 0:
                self.ax.scatter(*zip(*(np.array(collection.points))), **collection.kwargs)
        for collection in (self.scenes[self.i].lines + self.added_lines + self.added_rects):
            self.ax.add_collection(collection.get_collection())
        self.ax.autoscale(autoscaling)
        if not autoscaling:
            self.ax.set_xlim(xlim)
            self.ax.set_ylim(ylim)
        plt.draw()

In [ ]:
class Scene:
    def __init__(self, points=[], lines=[]):
        self.points=points
        self.lines=lines

class PointsCollection:
    def __init__(self, points, **kwargs):
        self.points = points
        self.kwargs = kwargs

    def add_points(self, points):
        self.points = self.points + points

class LinesCollection:
    def __init__(self, lines, **kwargs):
        self.lines = lines
        self.kwargs = kwargs

    def add(self, line):
        self.lines.append(line)

    def get_collection(self):
        return mcoll.LineCollection(self.lines, **self.kwargs)

class Plot:
    def __init__(self, scenes = [Scene()], points = [], lines = [], json = None):
        if json is None:
            self.scenes = scenes
            if points or lines:
                self.scenes[0].points = points
                self.scenes[0].lines = lines
        else:
            self.scenes = [Scene([PointsCollection(pointsCol) for pointsCol in scene["points"]],
                                 [LinesCollection(linesCol) for linesCol in scene["lines"]])
                           for scene in js.loads(json)]

    def __configure_buttons(self):
        plt.subplots_adjust(bottom=0.2)
        ax_prev = plt.axes([0.6, 0.05, 0.15, 0.075])
        ax_next = plt.axes([0.76, 0.05, 0.15, 0.075])
        ax_add_point = plt.axes([0.44, 0.05, 0.15, 0.075])
        ax_add_line = plt.axes([0.28, 0.05, 0.15, 0.075])
        ax_add_rect = plt.axes([0.12, 0.05, 0.15, 0.075])
        b_next = Button(ax_next, 'Następny')
        b_next.on_clicked(self.callback.next)
        b_prev = Button(ax_prev, 'Poprzedni')
        b_prev.on_clicked(self.callback.prev)
        b_add_point = Button(ax_add_point, 'Dodaj punkt')
        b_add_point.on_clicked(self.callback.add_point)
        b_add_line = Button(ax_add_line, 'Dodaj linię')
        b_add_line.on_clicked(self.callback.add_line)
        b_add_rect = Button(ax_add_rect, 'Dodaj figurę')
        b_add_rect.on_clicked(self.callback.add_rect)
        return [b_prev, b_next, b_add_point, b_add_line, b_add_rect]

    def add_scene(self, scene):
        self.scenes.append(scene)

    def add_scenes(self, scenes):
        self.scenes = self.scenes + scenes

    def toJson(self):
        return js.dumps([{"points": [np.array(pointCol.points).tolist() for pointCol in scene.points],
                          "lines":[linesCol.lines for linesCol in scene.lines]}
                         for scene in self.scenes])

    def get_added_points(self):
        if self.callback:
            return self.callback.added_points
        else:
            return None

    def get_added_lines(self):
        if self.callback:
            return self.callback.added_lines
        else:
            return None

    def get_added_figure(self):
        if self.callback:
            return self.callback.added_rects
        else:
            return None

    def get_added_elements(self):
        if self.callback:
            return Scene(self.callback.added_points, self.callback.added_lines+self.callback.added_rects)
        else:
            return None

    def draw(self):
        plt.close()
        fig = plt.figure()
        self.callback = _Button_callback(self.scenes)
        # self.widgets = self.__configure_buttons()
        ax = plt.axes(autoscale_on = False)
        self.callback.set_axes(ax)
        fig.canvas.mpl_connect('button_press_event', self.callback.on_click)
        plt.show()
        self.callback.draw()

# ROZWIĄZANIE LAB1

### Zbiór 1: 10^5^ losowych punktów z przedziału [-1000, 1000]

In [ ]:
%matplotlib notebook

points_a = [(random.uniform(-1000, 1000), random.uniform(-1000, 1000)) for _ in range(pow(10, 5))]

plot_a = Plot(points=[PointsCollection(points_a)])
plot_a.draw()


### Zbiór 2: 10^5^ losowych punktów z przedziału [-10^14^, 10^14^]

In [ ]:
%matplotlib notebook

m = pow(10, 14)
points_b = [(random.uniform(-m, m), random.uniform(-m, m)) for _ in range(pow(10, 5))]

plot_b = Plot(points=[PointsCollection(points_b)])
plot_b.draw()

### Zbiór 3: 1000 losowych punktów na okręgu o środku (0, 0) i promieniu R = 100

In [ ]:
%matplotlib notebook

from math import sin, cos, pi
def circle_point(t, R):
    return R*sin(2*pi*t), R*cos(2*pi*t)

R = 100
circle = [circle_point(random.uniform(0, 1), R) for _ in range(1000)]
circle_plot = Plot(points=[PointsCollection(circle)])
circle_plot.draw()

### Zbiór 4: 1000 losowych punktów leżących na prostej wyznaczonej przez wektor (a, b)

In [ ]:
%matplotlib notebook

A = (-1, 0)
B = (1, 0.1)
w_a = (A[1] - B[1]) / (A[0] - B[0])
w_b = A[1] - w_a*A[0]
# Równanie prostej (a, b)
def vector(x):
    return x, w_a*x + w_b

points_line = [vector(random.uniform(-1000, 1000)) for _ in range(1000)]
line_plot = Plot(points=[PointsCollection(points_line)])
line_plot.draw()

### Implementacje obliczania wyznaczników 2x2 oraz 3x3

In [ ]:
def det_1(c):
    return (A[0] - c[0])*(B[1] - c[1]) - (B[0] - c[0])*(A[1] - c[1])

# Metoda Sarrusa
def det_2(c):
    return (A[0]*B[1] + A[1]*c[0] + B[0]*c[1] -
            c[0]*B[1] - A[0]*c[1] - B[0]*A[1])


### Funkcje do obliczania wyznacznikow 2x2 oraz 3x3 z wykorzystaniem numpy

In [ ]:
def det_3(c):
    return np.linalg.det(np.array([[A[0] - c[0], A[1] - c[1]], [B[0] - c[0], B[1] - c[1]]]))

def det_4(c):
    return np.linalg.det(np.array([[A[0], A[1], 1], [B[0], B[1], 1], [c[0], c[1], 1]]))

### Funkcja rozdzielająca punkty według położenia względem prostej


In [ ]:
# Funckja przyjmuje zbiór punktów według, którego należy dokonać podziału,
# epsilon określający tolerancje dla zera
# oraz funckje lambda det, która oblicza wyznacznik według określonej metody
# parametry prostej są globalne
%matplotlib notebook
def categorize(points, det, eps):
    left, right, line = [], [], []
    for point in points:
        d = det(point)
        if abs(d) < eps:
            line.append(point)
        elif d > 0:
            left.append(point)
        else:
            right.append(point)

    return Plot(points=[PointsCollection(left, color="green"), PointsCollection(line, color="red"), PointsCollection(right, color="blue")]), len(left), len(right), len(line)

### Funkcja do znajdywania różnic w wykresach

In [ ]:
def compare(points, dets, eps):
    dif = []
    for point in points:
        values = [det(point) for det in dets]
        for i in range(len(dets)):
            if abs(values[i]) < eps:
                values[i] = 0
            elif values[i] > 0:
                values[i] = 1
            else:
                values[i] = -1
        for i in range(1, len(dets)):
            if values[i] != values[i-1]:
                dif.append(point)
                break
    return Plot(points=[PointsCollection(dif, color="blue")])



### Porównanie wyznaczników i tolerancji dla zbioru A

In [ ]:
%matplotlib notebook
tolerance = [pow(10, -12), pow(10, -14), pow(10, -16)]
functions = [lambda x: det_1(x), lambda x: det_2(x), lambda x: det_3(x), lambda x: det_4(x)]
for foo in functions:
    for eps in tolerance:
        plot, l, r, li = categorize(points_a, foo, eps)
        print(l, r, li)

categorize(points_a, functions[0], tolerance[0])[0].draw()


In [ ]:
%matplotlib notebook
for foo in functions:
    for eps in tolerance:
        plot, l, r, li = categorize(points_b, foo, eps)
        print(l, r, li)

categorize(points_b, functions[0], tolerance[0])[0].draw()

In [ ]:
%matplotlib notebook
plot, l, r, li = categorize(points_b, functions[0], tolerance[2])
plot.draw()
print(l, r, li)

In [ ]:
%matplotlib notebook
for foo in functions:
    for eps in tolerance:
        plot, l, r, li = categorize(circle, foo, eps)
        print(l, r, li)

categorize(circle, functions[0], tolerance[0])[0].draw()

In [ ]:
%matplotlib notebook
for foo in functions:
    for eps in tolerance:
        plot, l, r, li = categorize(points_line, foo, eps)
        print(l, r, li)

categorize(points_line, functions[0], tolerance[0])[0].draw()

In [ ]:
%matplotlib notebook

categorize(points_line, functions[0], tolerance[2])[0].draw()

In [ ]:
%matplotlib notebook
compare(points_line, functions, tolerance[1]).draw()

In [ ]:
%matplotlib notebook
compare(points_line, [functions[2], functions[3]], tolerance[1]).draw()

In [ ]:
%matplotlib notebook
for foo in functions:
    plot, l, r, li = categorize(points_line, foo, pow(10, -10))
    print(l, r, li)

In [ ]:
from time import time
points_1 = [(random.uniform(-1000, 1000), random.uniform(-1000, 1000)) for _ in range(pow(10, 5))]
points_2 = [(random.uniform(-1000, 1000), random.uniform(-1000, 1000)) for _ in range(5*pow(10, 5))]
points_3 = [(random.uniform(-1000, 1000), random.uniform(-1000, 1000)) for _ in range(pow(10, 6))]

sets = [points_1, points_2, points_3]
t = [[0 for _ in range(4)] for _ in range(3)]
for _ in range(3):
    i = 0
    for p in sets:
        j = 0
        for foo in functions:
            t1 = time()
            categorize(p, foo, tolerance[0])
            t2 = time()
            t[i][j] += (t2 - t1)
            j += 1
        i += 1

for i in range(3):
    for j in range(4):
        print(round(t[i][j] / 3, 4))